# Dados
Os dados utilizados neste notebook podem ser encontrados em https://drive.google.com/drive/u/1/folders/0ALqN9KkemfmvUk9PVA

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.image as image
from time import time
import imageio as io
import random
import pandas as pd
from sklearn.neural_network import MLPRegressor



# Funções:

In [2]:
def sigmoid(x, y, b, val):    #Função de ativação usada para avaliar a rede

    z = np.array(np.dot(y, x),dtype=np.float32)
    
    if val:
        z = z.transpose() + b
    else:
        z += b

    a = (1/(1+np.exp(-z)))
            
    #a = np.array(a)

    return a, z

def softmax(x, y, b, val):

    z = np.dot(y, x)
    
    if val:
        output = []
        z = z.transpose() + b
        for exp in np.exp(z):
            output.append(exp / np.sum(exp))
            len(exp)
        np.array(output)
    else:
        z += b
        exp = np.exp(z)
        output = exp / np.sum(exp)

    return output, z

def tanh(x, y, b):
    z = np.dot(y, x) + b
    a = []
    for h in z:
        if h >= 700:
            a.append(1)

        elif h <= -700:
            a.append(0)

        else:
            a.append(np.tanh(h))
            
    a = np.array(a)

    return a, z

def derivate_sigmoid(X):  #Derivada da função de ativação

    exp = np.exp(-X)
    
    der = (1/(1 + exp) * (1 - 1/(1 + exp)))

                       
    return der


def derivate_tanh(X):  #Derivada da função de ativação
    der = []
    for x in X:
        
        if x < 700 and x > -700:   # Condição usada para não dar erro, mas espera-se que os valores nunca estejam fora desse intervalo, já que o neurônio ficaria saturado
            der.append(1-np.tanh(x)**2)
                       
        else:
            der.append(0)
                       
    return np.array(der)




class Network():
    def __init__(self, architecture):   # Aqui, serão iniciados aleatóriamente os parâmetros da rede, conforme estrutura informada em "architeture"
        self.architecture = architecture
        self.w = []
        self.b = []
        for i in np.arange(1,len(architecture)):
            self.w.append([])
            self.b.append([])
            for j in range(architecture[i]):
                self.b[i-1].append(np.random.normal(0,1))
                self.w[i-1].append([])
                for k in range(architecture[i-1]):
                    self.w[i-1][j].append(np.random.normal(0,1/np.sqrt(architecture[i-1])))  # Este desvio padrão é usado para previnir que os neurônios estejam saturados no começo
            self.w[i-1] = np.array(self.w[i-1])
            self.b[i-1] = np.array(self.b[i-1])
            
    def cost_function(self, x, y):    # x -> output; y -> input 
        
        n = len(y)
        a, trash = self.evaluate_2(np.array(x))
        a = a[-1]
        #
        #(np.array(y) - a)**2
        C = -np.sum(np.array(y) * np.log(np.array(a))) / (2 * n)

        return C


    def evaluate(self, y):      # Avalia a rede neural
        a = [y]
        z = []
        for i in range(len(self.w)):
            
            if i == len(self.w) - 1:
                vec_aux = softmax(a[i], self.w[i], self.b[i], False)
            else:
                vec_aux = sigmoid(a[i], self.w[i], self.b[i], False)
                
            z.append(vec_aux[1])                  
            a.append(vec_aux[0])

            
        return a, z                # A saída mais importante é o "a", mas o "z" foi acrescentado aqui em uma tentativa de otimização
    
    
    def evaluate_2(self, y):      # Avalia a rede neural
        a = [y]
        z = []
        for i in range(len(self.w)):
            
            if i == len(self.w) - 1:
                vec_aux = softmax(a[i].transpose(), self.w[i], self.b[i], True)
            else:
                vec_aux = sigmoid(a[i].transpose(), self.w[i], self.b[i], True)
                
            z.append(vec_aux[1])                  
            a.append(vec_aux[0])

            
        return a, z               


    def update(self, y, x, eta9, nu):           # Função usada para atualizar os pesos, W's, e bias, b's 
        
        eta = -eta9  # Learning rate
        
        #dw, db = derivative(y, self, x, self.w, self.b)
        
        #------------------------------------------------------------------------------------------------
        
        n = len(y)


        dw = []
        db = []
        
        for i in range(len(self.w)):
            dw.append(0)
            db.append(0)

        a, z = self.evaluate_2(np.array(y))  # Avalia os dados de entrada na rede e devolve os dados de saída, "a", e o "z", que é usado no processo de avaliação    
                
        for l in range(n): #while l < n:           
            
            for i in range(len(self.w)):   # Rodando em todas as camadas da rede
                
                i = len(self.w) - i - 1    # O algorítmo do 'Back Propagation' começa no fim e termina no começo
                
                da_aux = 0


                d_sigma = derivate_sigmoid(z[i][l]) 

  
                '''
                ---------------------------------------------------------------------------------------------------------------------
                Aqui é calculado a derivada da função de custo em relação aos a's - matriz representando a saída de cada neurônio da rede
                
                '''

                if i == len(self.w) - 1 :
                    
                    da_aux = np.dot((a[i + 1][l] - x[l]), self.w[i])
                        
                else:
                    
                    da_aux = np.dot(da * d_sigma, self.w[i])
                    
                '''
                ---------------------------------------------------------------------------------------------------------------------
                Aqui é calculado a derivada da função de custo em relação aos w's - pesos dos neurônios
                
                '''
                        

                if i == len(self.w) - 1 :
                    
                    dw_aux = np.outer((a[i + 1][l] - x[l]), a[i][l]) + nu * self.w[i]

                else:
                    
                    dw_aux = np.outer(da * d_sigma, a[i][l]) + nu * self.w[i] 
                    
                '''
                ---------------------------------------------------------------------------------------------------------------------
                Aqui é calculado a derivada da função de custo em relação aos b's
                
                '''

                if i == len(self.w) - 1:              

                    db_aux = (a[i + 1][l] - x[l])
                    
                else:      
                    db_aux = da * d_sigma
                    
                # O resultado é, então, acresentado em dw e db, representando a derivada total na época em questão

                dw[i] += eta * np.array(dw_aux)
                db[i] += eta * np.array(db_aux)
                da = da_aux
  
        #------------------------------------------------------------------------------------------------
            
        for m in range(len(self.w)):   # Os w's e b's são atualizados
            
            self.w[m] = self.w[m] + dw[m] / n
            self.b[m] = self.b[m] + db[m] / n

                    
        return None
    
    def run_epoch(self, learning_rate, nu, times_interaction, Training_input, training_output):    
        eta = learning_rate                     # Taxa de aprendizagem (Learning rate)
                            # Regularização
        graph = False              # Informa se será plotado gráficos da função de custo
        minibatch = 0            # length of the minibatch. If 0, all the integer batch is going to be used

        if minibatch != 0:    # Necessário para fazer o random choice
            dataset = {}
            dataset['input'] = Training_input
            dataset['output'] = training_output
            dataset = pd.DataFrame(dataset)
            index = np.arange(0,len(Training_input)).tolist()

        if graph:             # Dados para o ponto de interação zero do gráfico

            Training_cost = [self.cost_function(training_output, Training_input)]       
            Validation_cost = [self.cost_function(validation_output, validation_input)]

        start = time()
        for i in range(times_interaction):

            if minibatch != 0:  #Random choice do minibatch

                chose_index = random.sample(index, minibatch)
                Training_input_aux = dataset.loc[chose_index, 'input'].values.tolist()
                training_output_aux = dataset.loc[chose_index, 'output'].values.tolist()
                self.update(Training_input_aux, training_output_aux, eta, nu)

            else:    

                self.update(Training_input, training_output, eta, nu)

            if graph:
                Training_cost.append(self.cost_function(training_output, Training_input))
                Validation_cost.append(self.cost_function(validation_output, validation_input))

            if i == 0:           # Informa o tempo que levará para acabar
                stop = time()
                print(str(-start * times_interaction/3600 + stop * times_interaction/3600) + 'h para finalizar')


        if times_interaction > 0 and graph:
            t = np.arange(0, times_interaction+1)
            plt.plot(t, Training_cost, '.', label = 'Training')
            plt.plot(t, Validation_cost, '.', label = 'Validation')
            plt.xlabel('Learning Interaction')
            plt.ylabel('Cost Function')
            plt.title('Cost Function Evolution')
            plt.legend()
            plt.show()
        return None
        
def test(net, validation_input, validation_output):
    cont = 0
    result_output, trash = net.evaluate_2(np.array(validation_input))
    for i in range(len(validation_input)):
        
        test = interpretation(result_output[-1][i], validation_output[i])

        if test:
            cont += 1
    return cont/len(validation_output)*100


def saving(net):
    for i in range(len(net.w)):
        file = open(str(i) + '.txt', 'w')
        for j in range(len(net.w[i])):
            for k in range(len(net.w[i][j])):
                file.write(str(net.w[i][j][k]) + ';')
            file.write('\n')




        file.close()
    file = open('b' + '.txt', 'w')
    for i in range(len(net.b)):
        for j in range(len(net.b[i])):
            file.write(str(net.b[i][j]) + ';')
        file.write('\n')
    file.close()
    
    return None

def recovering(net):
    for i in range(len(net.w)):
        file = open(str(i) + '.txt', 'r')
        line = file.readline()
        j = 0
        while line != '':
            aux = ''
            k = 0
            for c in line:
                if c != ';' and c != '\n':
                    aux += c
                elif c != '\n':
                    net.w[i][j][k] = float(aux)
                    aux = ''
                    k += 1
            j += 1
            line = file.readline()


    file = open('b' + '.txt', 'r')
    line = file.readline()
    i = 0
    while line != '':
        aux = ''
        j = 0
        for c in line:
            if c != ';' and c != '\n':
                aux += c
            elif c != '\n':
                net.b[i][j] = float(aux)
                aux = ''
                j += 1
        i += 1
        line = file.readline()
    return None


def interpretation(y, x):
    t = 0
    length = abs(y[0] - 1)
    for b in range(len(y)):
        if abs(y[b] - 1) < length:
            t = b
            length = abs(y[b] - 1)
            
    k = np.zeros(10)
    k[t] = 1
    aux = 1
    for i in range(len(x)):
        if x[i] != k[i]:
            aux = 0
        
    return aux


* Carregando dados

In [3]:
path = os.getcwd()
name = 'heart_disease_health_indicators_BRFSS2015.csv'
file = os.path.join(path, name)
df = pd.read_csv(file, sep = ',')
key = 'BMI'
df.loc[:, key] = df.loc[:, key] / (df.loc[:, key].max() - df.loc[:, key].min()) - df.loc[:, key].min() / (df.loc[:, key].max() - df.loc[:, key].min())

del df['NoDocbcCost']
del df['MentHlth']
del df['PhysHlth']
del df['GenHlth']
del df['Income']
del df['Education']

key = 'Age'
df.loc[:, key] = df.loc[:, key] / (df.loc[:, key].max() - df.loc[:, key].min()) - df.loc[:, key].min() / (df.loc[:, key].max() - df.loc[:, key].min())

#Training_input = df[:]
training_output = df['HeartDiseaseorAttack'].values.tolist()[:int(len(df['HeartDiseaseorAttack']) * 0.9)]
validation_output = df['HeartDiseaseorAttack'].values.tolist()[int(len(df['HeartDiseaseorAttack']) * 0.9):]
for val in range(len(training_output)):
    if training_output[val] == 0:
        training_output[val] = [1, 0]
    elif training_output[val] == 1:
        training_output[val] = [0, 1]
    else:
        print('erro')
    #training_output[val] = [training_output[val]]
for val in range(len(validation_output)):
    if validation_output[val] == 0:
        validation_output[val] = [1, 0]
    elif validation_output[val] == 1:
        validation_output[val] = [0, 1]
    else:
        print('erro')
    #validation_output[val] = [validation_output[val]]
Training_input = []
validation_input = []
for i in range(int(len(df['HeartDiseaseorAttack']) * 0.9)):
    Training_input.append(df.iloc[i].values[1:])
    
for i in np.arange(int(len(df['HeartDiseaseorAttack']) * 0.9), len(df['HeartDiseaseorAttack'])):
    validation_input.append(df.iloc[i].values[1:])


In [7]:
len(df.keys())

16

In [137]:
path = os.getcwd()
name = 'cleveland.txt'
file = os.path.join(path, name)
df = pd.read_csv(file, sep = ',')
key = 'age'
df.loc[:, key] = df.loc[:, key] / (df.loc[:, key].max() - df.loc[:, key].min()) - df.loc[:, key].min() / (df.loc[:, key].max() - df.loc[:, key].min())
key = 'cp'
df.loc[:, key] = df.loc[:, key] / (df.loc[:, key].max() - df.loc[:, key].min()) - df.loc[:, key].min() / (df.loc[:, key].max() - df.loc[:, key].min())
key = 'trestbps'
df.loc[:, key] = df.loc[:, key] / (df.loc[:, key].max() - df.loc[:, key].min()) - df.loc[:, key].min() / (df.loc[:, key].max() - df.loc[:, key].min())
key = 'chol'
df.loc[:, key] = df.loc[:, key] / (df.loc[:, key].max() - df.loc[:, key].min()) - df.loc[:, key].min() / (df.loc[:, key].max() - df.loc[:, key].min())
key = 'restecg'
df.loc[:, key] = df.loc[:, key] / (df.loc[:, key].max() - df.loc[:, key].min()) - df.loc[:, key].min() / (df.loc[:, key].max() - df.loc[:, key].min())
key = 'thalach'
df.loc[:, key] = df.loc[:, key] / (df.loc[:, key].max() - df.loc[:, key].min()) - df.loc[:, key].min() / (df.loc[:, key].max() - df.loc[:, key].min())
key = 'oldpeak'
df.loc[:, key] = df.loc[:, key] / (df.loc[:, key].max() - df.loc[:, key].min()) - df.loc[:, key].min() / (df.loc[:, key].max() - df.loc[:, key].min())
key = 'slope'
df.loc[:, key] = df.loc[:, key] / (df.loc[:, key].max() - df.loc[:, key].min()) - df.loc[:, key].min() / (df.loc[:, key].max() - df.loc[:, key].min())
key = 'thal'
df.loc[:, key] = df.loc[:, key] / (df.loc[:, key].max() - df.loc[:, key].min()) - df.loc[:, key].min() / (df.loc[:, key].max() - df.loc[:, key].min())

     
    
#Training_input = df[:]
training_output = df['the predicted attribute'].values.tolist()[:int(len(df['the predicted attribute']) * 0.9)]
validation_output = df['the predicted attribute'].values.tolist()[int(len(df['the predicted attribute']) * 0.9):]
for val in range(len(training_output)):
    if training_output[val] == 0:
        training_output[val] = [1, 0]
    else:
        training_output[val] = [0, 1]

    #training_output[val] = [training_output[val]]
for val in range(len(validation_output)):
    if validation_output[val] == 0:
        validation_output[val] = [1, 0]
    else:
        validation_output[val] = [0, 1]
        
Training_input = []
validation_input = []
for i in range(int(len(df['the predicted attribute']) * 0.9)):
    Training_input.append(df.iloc[i].values[:14])
    
for i in np.arange(int(len(df['the predicted attribute']) * 0.9), len(df['the predicted attribute'])):
    validation_input.append(df.iloc[i].values[:14])
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,the predicted attribute
0,0.708333,1.0,0.000000,0.481132,0.244292,1.0,1.0,0.603053,0.0,0.370968,1.0,0.0,0.75,0
1,0.791667,1.0,1.000000,0.622642,0.365297,0.0,1.0,0.282443,1.0,0.241935,0.5,3.0,0.00,2
2,0.791667,1.0,1.000000,0.245283,0.235160,0.0,1.0,0.442748,1.0,0.419355,0.5,2.0,1.00,1
3,0.166667,1.0,0.666667,0.339623,0.283105,0.0,0.0,0.885496,0.0,0.564516,1.0,0.0,0.00,0
4,0.250000,0.0,0.333333,0.339623,0.178082,0.0,1.0,0.770992,0.0,0.225806,0.0,0.0,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,0.583333,0.0,1.000000,0.433962,0.262557,0.0,0.0,0.396947,1.0,0.032258,0.5,0.0,1.00,1
293,0.333333,1.0,0.000000,0.150943,0.315068,0.0,0.0,0.465649,0.0,0.193548,0.5,0.0,1.00,1
294,0.812500,1.0,1.000000,0.471698,0.152968,1.0,0.0,0.534351,0.0,0.548387,0.5,2.0,1.00,2
295,0.583333,1.0,1.000000,0.339623,0.011416,0.0,0.0,0.335878,1.0,0.193548,0.5,1.0,1.00,3


In [106]:
path = os.getcwd()
name = 'heart.csv'
file = os.path.join(path, name)
df = pd.read_csv(file, sep = ',')
df.loc[:, 'Age'] = (df.loc[:, 'Age'] - 50)/ 100
df.loc[:, 'RestingBP'] = (df.loc[:, 'RestingBP'] - 100) / 200
df.loc[:, 'Cholesterol'] = (df.loc[:, 'Cholesterol'] - 100) / 600
df.loc[:, 'MaxHR'] = (df.loc[:, 'MaxHR'] - 100) / 200
df.loc[:, 'Oldpeak'] = (df.loc[:, 'Oldpeak']) / 10

for s in range(len(df.loc[:, 'Sex'])):
    if df.loc[s, 'Sex'] == 'M':
        df.loc[s, 'Sex'] = 1
    elif df.loc[s, 'Sex'] == 'F':
        df.loc[s, 'Sex'] = 0
    else:
        print( 'erro - ' + str(s))
        
    if df.loc[s, 'RestingECG'] == 'Normal':
        df.loc[s, 'RestingECG'] = -1
    elif df.loc[s, 'RestingECG'] == 'ST':
        df.loc[s, 'RestingECG'] = 1
    elif df.loc[s, 'RestingECG'] == 'LVH':
        df.loc[s, 'RestingECG'] = 0
    else:
        print( 'erro - ' + str(s))
        
    if df.loc[s, 'ExerciseAngina'] == 'Y':
        df.loc[s, 'ExerciseAngina'] = 1
    elif df.loc[s, 'ExerciseAngina'] == 'N':
        df.loc[s, 'ExerciseAngina'] = 0
    else:
        print( 'erro - ' + str(s))
        
    if df.loc[s, 'ST_Slope'] == 'Up':
        df.loc[s, 'ST_Slope'] = 1
    elif df.loc[s, 'ST_Slope'] == 'Flat':
        df.loc[s, 'ST_Slope'] = 0
    elif df.loc[s, 'ST_Slope'] == 'Down':
        df.loc[s, 'ST_Slope'] = -1
    else:
        print( 'erro - ' + str(s))
        
    if df.loc[s, 'ChestPainType'] == 'ASY':
        df.loc[s, 'ChestPainType'] = -1
    elif df.loc[s, 'ChestPainType'] == 'NAP':
        df.loc[s, 'ChestPainType'] = -2/3
    elif df.loc[s, 'ChestPainType'] == 'TA':
        df.loc[s, 'ChestPainType'] = 2/3
    elif df.loc[s, 'ChestPainType'] == 'ATA':
        df.loc[s, 'ChestPainType'] = 1
    else:
        print( 'erro - ' + str(s))
        
    
#Training_input = df[:]
training_output = df['HeartDisease'].values.tolist()[:int(len(df['HeartDisease']) * 0.9)]
validation_output = df['HeartDisease'].values.tolist()[int(len(df['HeartDisease']) * 0.9):]
for val in range(len(training_output)):
    if training_output[val] == 0:
        training_output[val] = [1, 0]
    elif training_output[val] == 1:
        training_output[val] = [0, 1]
    else:
        print('erro')
    #training_output[val] = [training_output[val]]
for val in range(len(validation_output)):
    if validation_output[val] == 0:
        validation_output[val] = [1, 0]
    elif validation_output[val] == 1:
        validation_output[val] = [0, 1]
    else:
        print('erro')
    #validation_output[val] = [validation_output[val]]
Training_input = []
validation_input = []
for i in range(int(len(df['HeartDisease']) * 0.9)):
    Training_input.append(df.iloc[i].values[:11])
    
for i in np.arange(int(len(df['HeartDisease']) * 0.9), len(df['HeartDisease'])):
    validation_input.append(df.iloc[i].values[:11])


In [112]:
len(df.keys())


12

In [9]:
def bla(validation_input, validation_output, net):
    cont = 0
    for i in range(len(validation_input)):
        result_output = rna.predict([validation_input[i]])
        test = interpretation(result_output[0], validation_output[i])
        if test:
            cont += 1
    return (str(cont/len(validation_output)*100) + '% de acertos')

90.43677073478398% de acertos


In [ ]:
rna = MLPRegressor(hidden_layer_sizes = (3), verbose = True, activation = 'logistic', solver = 'sgd', alpha = 0, learning_rate = 'constant', learning_rate_init = 0.1, tol = 0.00001, max_iter = 100)
rna.fit(Training_input, training_output)
for i in np.arange(2,100):

    rna = MLPRegressor(hidden_layer_sizes = (3), verbose = False, activation = 'logistic', solver = 'sgd', alpha = 0, learning_rate = 'constant', learning_rate_init = 1, tol = 0.00001, max_iter = 10)



    rna.fit(Training_input, training_output)
    print(bla(validation_input, validation_output, net))
    print(bla(Training_input, training_output, net))

    print('\n')

In [4]:
net = Network((len(Training_input[0]), 3, 2))
print(test(net, validation_input, validation_output))
print(test(net, Training_input, training_output))
print(net.cost_function(Training_input, training_output))

9.748502049826554
9.381898454746135
0.8552932341901357


In [5]:
learning_rate = 0.01
epochs = 10
regularization_parameter = 0
net.run_epoch(learning_rate, regularization_parameter, epochs, Training_input, training_output)
print(test(net, validation_input, validation_output))
print(test(net, Training_input, training_output))
print(net.cost_function(Training_input, training_output))

0.11160421278327703h para finalizar
9.748502049826554
9.381898454746135
0.7332561377377657


In [24]:
0.17833757990981924

0.17833757990981924

In [ ]:
for i in np.arange(2,100):
    learning_rate = 1
    net = Network((len(Training_input[0]), i, 2))
    epochs = 10
    regularization_parameter = 0
    net.run_epoch(learning_rate, regularization_parameter, epochs, Training_input, training_output)
    print(test(net, validation_input, validation_output))
    print(test(net, Training_input, training_output))
    print(net.cost_function(Training_input, training_output))
    print('\n')

0.11202501319348812h para finalizar
90.25149795017344
90.61810154525386
0.15168647700683577


0.1040640352293849h para finalizar
90.25149795017344
90.61810154525386
0.14459456026465617


0.1082666078582406h para finalizar
90.25149795017344
90.61810154525386
0.15430368019070995


0.10771681554615498h para finalizar
90.25149795017344
90.61810154525386
0.15835798775526913


0.10430266335606575h para finalizar
90.25149795017344
90.61810154525386
0.15163510005287725


0.10495846252888441h para finalizar
90.25149795017344
90.61810154525386
0.15460309049633522


0.10576672200113535h para finalizar
90.25149795017344
90.61810154525386
0.15266214288467736


0.10696111805737019h para finalizar
90.25149795017344
90.61810154525386
0.15714761476152891


0.1063471082597971h para finalizar
90.25149795017344
90.61810154525386
0.15308110646280854


0.1075584813952446h para finalizar
90.25149795017344
90.61810154525386
0.15220806194831346


0.10895292367786169h para finalizar
90.25149795017344
90.6181015

In [ ]:
nets = []
num_net = 20
for i in range(num_net):
    nets.append(Network((len(validation_input[0]), 101, 10)))

testing = np.zeros(num_net).tolist()
fitters = []
for j in range(num_net):
    
    C = nets[j].cost_function(Training_input, training_output)

    TF_value = (C > np.array(testing))
    if False in TF_value:
        aux = True
        for v in range(len(TF_value)):
            if (not TF_value[v]) and aux:
                fitters = fitters[0:v] + [nets[j]] + fitters[v:]
                testing = testing[0:v] + [C] + testing[v:len(testing) - 1]
                aux = False
        '''net_1 = np.random.choice(nets)
        net_2 = np.random.choice(nets)
        nets[j] = crossing(net_1, net_2)'''
    else:
        testing[j] = C
        fitters.append(nets[j])
            
nets = fitters   

In [ ]:
def crossing(net_1, net_2):
    
    net = Network((len(validation_input[0]), 101, 10))
    mutation = 0.000001
    
    for i in range(len(net.w)):
        #trashold_b = random.randint(0, len(net.b[i]) - 1)
        for j in range(len(net.w[i])):
            
            if random.random() > mutation:
                if random.random() < 0.5:
                    net.b[i][j] = net_1.b[i][j]
                else:
                    net.b[i][j] = net_2.b[i][j]
                
            #trashold_w = random.randint(0, len(net.w[i][j]) - 1)
            
            for k in range(len(net.w[i][j])):
                
                if random.random() > mutation:
                    if random.random() < 0.5:
                        net.w[i][j][k] = net_1.w[i][j][k]
                    else:
                        net.w[i][j][k] = net_2.w[i][j][k]
                
    return net
      
times_interaction = 500
num_net = len(nets)
start = time()

for j in range(times_interaction):

    net_1 = np.random.choice(nets[:int(len(nets)/2)])
    net_2 = np.random.choice(nets[:int(len(nets)/2)])
    net = crossing(net_1, net_2)
    C = net.cost_function(Training_input, training_output)
    aux = True
    i = 0
    while i < len(nets) and aux:
        if C < testing[i]:

            nets = nets[:i] + [net] + nets[i:]
            nets = nets[:len(nets) - 1]
            testing = testing[:i] + [C] + testing[i:]
            testing = testing[:len(testing) - 1]

            aux = False
        i += 1
    if j == 0:
        stop = time()

        print(str((stop - start) * times_interaction / 60) + ' min para finalizar')

In [ ]:
for j in range(len(nets)):        
    print(nets[j].cost_function(Training_input, training_output))

In [ ]:
for net in nets:        
    print(test(net, validation_input, validation_output))

In [124]:
df['Age'].min()


-0.22